In [1]:
!pip install ortools

   ---------------------------------------- 0.0/20.5 MB ? eta -:--:--
   -- ------------------------------------- 1.0/20.5 MB 9.2 MB/s eta 0:00:03
   ---- ----------------------------------- 2.1/20.5 MB 10.0 MB/s eta 0:00:02
   ---- ----------------------------------- 2.1/20.5 MB 10.0 MB/s eta 0:00:02
   ---- ----------------------------------- 2.1/20.5 MB 10.0 MB/s eta 0:00:02
   ------ --------------------------------- 3.1/20.5 MB 3.2 MB/s eta 0:00:06
   ------ --------------------------------- 3.1/20.5 MB 3.2 MB/s eta 0:00:06
   ------ --------------------------------- 3.4/20.5 MB 2.2 MB/s eta 0:00:08
   -------- ------------------------------- 4.2/20.5 MB 2.8 MB/s eta 0:00:06
   ---------- ----------------------------- 5.2/20.5 MB 2.7 MB/s eta 0:00:06
   ------------ --------------------------- 6.3/20.5 MB 3.0 MB/s eta 0:00:05
   ------------ --------------------------- 6.3/20.5 MB 3.0 MB/s eta 0:00:05
   -------------- ------------------------- 7.3/20.5 MB 3.0 MB/s eta 0:00:05
   


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ortools.linear_solver import pywraplp

In [3]:
def ler_dados_entrada(arquivo):
    """
    
    
    Formato esperado do arquivo:
    Linha 1: tamanho da barra original
    Linha 2: quantidade de tipos de itens
    Linha 3: tamanhos dos itens (separados por espaço)
    Linha 4: demandas dos itens (separados por espaço)
    
    """
    with open(arquivo, 'r') as f:
        linhas = f.readlines()
    
    tamanho_barra = int(linhas[0].strip())
    n_tipos = int(linhas[1].strip())
    tamanhos = list(map(int, linhas[2].strip().split()))
    demandas = list(map(int, linhas[3].strip().split()))
    
    return tamanho_barra, tamanhos, demandas


arquivo_entrada = 'entrada.txt'  


tamanho_barra, tamanhos, demandas = ler_dados_entrada(arquivo_entrada)
print(f"Dados lidos do arquivo '{arquivo_entrada}'")

print(f"\nTamanho da barra original: {tamanho_barra}m")
print(f"Tipos de itens: {len(tamanhos)}")
for i, (tam, dem) in enumerate(zip(tamanhos, demandas)):
    print(f"  Item {i+1}: {dem} barras de {tam}m")

Dados lidos do arquivo 'entrada.txt'

Tamanho da barra original: 150m
Tipos de itens: 3
  Item 1: 70 barras de 80m
  Item 2: 100 barras de 60m
  Item 3: 120 barras de 50m


## Geração de Padrões de Corte


In [ ]:
def gerar_padroes_corte_maximal(tamanho_barra, tamanhos):
    
    n_itens = len(tamanhos)
    padroes = []
    def gerar_recursivo(indice, espaco_restante, padrao_atual):
        if indice == n_itens:
            
            pode_melhorar = False
            for k in range(n_itens):
                if espaco_restante >= tamanhos[k]:
                    pode_melhorar = True
                    break
            if not pode_melhorar and sum(padrao_atual) > 0:
                padroes.append(padrao_atual[:])
            return
        max_quantidade = espaco_restante // tamanhos[indice]
        for quantidade in range(max_quantidade + 1):
            padrao_atual[indice] = quantidade
            novo_espaco = espaco_restante - (quantidade * tamanhos[indice])
            gerar_recursivo(indice + 1, novo_espaco, padrao_atual)
    padrao_inicial = [0] * n_itens
    gerar_recursivo(0, tamanho_barra, padrao_inicial)
    return padroes

In [ ]:
# Gera apenas os padrões de corte maximal
padroes = gerar_padroes_corte_maximal(tamanho_barra, tamanhos)

print(f"Total de padrões de corte gerados: {len(padroes)}\n")
print("Padrões de corte:")
print("-" * 80)

for i, padrao in enumerate(padroes):
    material_usado = sum(padrao[j] * tamanhos[j] for j in range(len(tamanhos)))
    desperdicio = tamanho_barra - material_usado
    # descrição do padrão como soma dos tamanhos
    descricao = []
    for j in range(len(tamanhos)):
        descricao.extend([str(tamanhos[j])] * padrao[j])
    descricao_str = " + ".join(descricao)
    print(f"p{i+1}: {descricao_str} = {material_usado}m usados (desperdício: {desperdicio}m)")
    print(f"     {padrao}")

Total de padrões de corte gerados: 5

Padrões de corte:
--------------------------------------------------------------------------------
p1: 50 + 50 + 50 = 150m usados (desperdício: 0m)
     [0, 0, 3]
p2: 60 + 50 = 110m usados (desperdício: 40m)
     [0, 1, 1]
p3: 60 + 60 = 120m usados (desperdício: 30m)
     [0, 2, 0]
p4: 80 + 50 = 130m usados (desperdício: 20m)
     [1, 0, 1]
p5: 80 + 60 = 140m usados (desperdício: 10m)
     [1, 1, 0]
